In [1]:
import h2o
import pandas as pd
from sklearn.metrics import accuracy_score

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,12 hours 14 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,"7 days, 16 hours and 45 minutes"
H2O cluster name:,H2O_from_python_emily_air13_v0k8iv
H2O cluster total nodes:,1
H2O cluster free memory:,1.580 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [4]:
model = h2o.load_model('saved_models/h2o_leader/GBM_1_AutoML_20190321_010832')

In [5]:
model.model_performance()


ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.16083738715028315
RMSE: 0.4010453679451779
LogLoss: 0.490819441597838
Mean Per-Class Error: 0.240199532881742
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



negative,neutral,positive,Error,Rate
83.0,44.0,30.0,0.4713376,74 / 157
0.0,706.0,132.0,0.1575179,132 / 838
0.0,60.0,594.0,0.0917431,60 / 654
83.0,810.0,756.0,0.1613099,"266 / 1,649"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.8386901
2,0.9775621
3,1.0


In [6]:
ge_data = h2o.import_file('data_files/GE_output.csv')

predictors = ['IbmScore', 'GCPScore', 'AzureScore', 'AWSPosScore', 'AWSNegScore', 'AWSNeuScore']
response = "ActualLabel"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
predictions = model.predict(ge_data)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [10]:
predictions

predict,negative,neutral,positive
positive,0.0832976,0.23979,0.676912
neutral,0.09598,0.7368,0.16722
positive,0.0956338,0.137474,0.766893
neutral,0.273228,0.430434,0.296339
positive,0.384386,0.168962,0.446652
neutral,0.0405986,0.775156,0.184245
neutral,0.1328,0.629633,0.237567
neutral,0.284464,0.415534,0.300002
positive,0.0564,0.286676,0.656924
neutral,0.224862,0.543016,0.232122


In [11]:
metrics = model.model_performance(ge_data)

In [12]:
metrics


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.3126006387401189
RMSE: 0.559107001154626
LogLoss: 0.8622143591522115
Mean Per-Class Error: 0.5818373117643191
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



negative,neutral,positive,Error,Rate
0.0,6.0,1.0,1.0,7 / 7
1.0,83.0,53.0,0.3941606,54 / 137
1.0,12.0,24.0,0.3513514,13 / 37
2.0,101.0,78.0,0.4088398,74 / 181


Top-3 Hit Ratios: 


k,hit_ratio
1,0.5911602
2,0.9558011
3,1.0


In [19]:
pdf = predictions.as_data_frame()
vd = pd.read_csv('data_files/GE_output.csv')

In [20]:
accuracy_score(vd['ActualLabel'], pdf['predict'])

0.5911602209944752